In [264]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
import string

# 預處理x
def preprocess_x(x):
    x = str(x) # 0.轉字串
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ') # 3.去除換行符號
    x = x.replace('x000D','') # 4.移除'x000D'
    x = ' ' + str.strip(x) + ' ' # 5.移除左右空白 在左右各加一格空白
    return x

# 預處理y
def preprocess_y(x):
    x = str(x) # 0.轉字串
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ') # 3.去除換行符號
    x = ' '.join([ i.replace(' ','') for i in x.split(' ')])
    x = str.strip(x) # 4.移除左右空白
    return x

# 人工規則找銀行
def get_bank(text,split_keywords,尾綴列表):
    text = preprocess_x(text)
    def 根據尾綴找答案(text,尾綴):
        if 尾綴 in text:
            result = preprocess_x(text[:text.find(尾綴)+len(尾綴)]).strip()
            if (尾綴 in result) and (len(result) > len(尾綴)):
                return result
            else:
                return 'not find'
        else:
            return 'not find'
    for keyword in split_keywords:
        if keyword in text:
            result_list = []
            for 尾綴 in 尾綴列表:
                result = 根據尾綴找答案(text.split(keyword)[1],尾綴)
                if result != 'not find':
                    return result
                else:
                    return 'not find'
        else:
            return 'not find'

# 根據寶典找銀行
def get_bank_寶典(x,寶典):
    for p in 寶典:
        if p in x:
            return p
    return 'not find'

# 找 Y1 Y2
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

In [265]:
# load raw data
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['46A','47A','78','LCBK']].dropna(subset=['LCBK']).astype(str)
print(df.shape)
df.head(10)


(9813, 4)


,46A,47A,78,LCBK
3,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA..."
4,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)"
5,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...,+ UPON RECEIPT OF ALL DOCUMENTS SENT TO US(VIE...,VIET NAM BANK FOR AGRICULTURE \nAND RURAL DEVE...
7,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
8,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
9,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
10,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
11,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
12,+1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL (S)...,+EACH SET OF DOCUMENTS PRESENTED WITH DISCREPA...,+ FORWARD DRAFTS AND DOCUMENTS TO STANDARD CHA...,STANDARD CHARTERED BANK (CHINA) \nLIMITED (HAN...
13,+1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL (S)...,+EACH SET OF DOCUMENTS PRESENTED WITH DISCREPA...,+ FORWARD DRAFTS AND DOCUMENTS TO STANDARD CHA...,STANDARD CHARTERED BANK (CHINA) \nLIMITED (HAN...


In [266]:
# X,Y製作
df['string_X_train'] = df['46A'] +df['47A'] +df['78']
df['string_X_train'] = df['string_X_train'].apply(lambda x:preprocess_x(x))
df['Y_label'] = df['LCBK'].apply(lambda x:preprocess_y(x))
銀行列表 = list(set(df['Y_label'].values.tolist()))
尾綴列表 = pd.DataFrame(df['Y_label'].apply(lambda x:str(x).split(' ')[-1]).values.tolist(),columns=['尾綴'])
尾綴列表 = 尾綴列表['尾綴'].value_counts().sort_values(ascending=False)
尾綴列表 = 尾綴列表[尾綴列表>1].index.tolist()[:5]
display(尾綴列表)
print(len(銀行列表))
try:
    for i in ['OF','THE']:        
        尾綴列表.remove(i)
except:
    pass
split_keywords = ['TO ORDER OF','TO THEORDER OF','TO THE ORDER OF','TOTHE ORDER OF','TO THE ORDER+OF','TOORDER OF']

# Y1,Y2製作
ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['string_X_train'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst

# 清洗不合格數據
print(df.shape)
df = df[df['string_Y_1'] != -1]
print(df.shape)
df = df[df['Y_label'] != '']
print(df.shape)
df = df[df['Y_label'] != str('nan')]
print(df.shape)
df.head(20)

['BANK', 'LTD', 'LIMITED', 'CHINA', 'BRANCH']

751
(9813, 8)
(3731, 8)
(3731, 8)
(3731, 8)


,46A,47A,78,LCBK,string_X_train,Y_label,string_Y_1,string_Y_2
16,+SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDI...,"+APPLICANT'S ADDRESS:19F, HAICANG BUSINESS BUI...",+TO NEGOTIATING BANK ONLY : PLEASE FORWARD ALL...,"CTBC BANK CO., LTD",SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDI...,CTBC BANK CO LTD,1860,1876
18,+ SIGNED COMMERCIAL INVOICE IN TRIPLICATE+ FUL...,+A DISCREPANCY FEE OF USD80.00(OR EQUIVALENT) ...,+ALL DOCUMENTS MUST BE FORWARDED DIRECTLY TO O...,"INDUSTRIAL BANK OF KOREA, SEOUL",SIGNED COMMERCIAL INVOICE IN TRIPLICATE FULL ...,INDUSTRIAL BANK OF KOREA SEOUL,874,904
19,+ SIGNED COMMERCIAL INVOICE IN TRIPLICATE+ FUL...,+A DISCREPANCY FEE OF USD80.00(OR EQUIVALENT) ...,+ALL DOCUMENTS MUST BE FORWARDED DIRECTLY TO O...,"INDUSTRIAL BANK OF KOREA, SEOUL",SIGNED COMMERCIAL INVOICE IN TRIPLICATE FULL ...,INDUSTRIAL BANK OF KOREA SEOUL,874,904
22,+SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICA...,+THIS L/C IS PAYABLE 30 DAYS AFTER B/L DATE (B...,+TO NEGOTIATING BANK ONLY: PLEASE FORWARD ALL ...,"CTBC BANK CO., LTD.",SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICA...,CTBC BANK CO LTD,4924,4940
23,+SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICA...,+THIS L/C IS PAYABLE 30 DAYS AFTER B/L DATE (B...,+TO NEGOTIATING BANK ONLY: PLEASE FORWARD ALL ...,"CTBC BANK CO., LTD.",SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICA...,CTBC BANK CO LTD,4924,4940
24,+SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICA...,+THIS L/C IS PAYABLE 30 DAYS AFTER B/L DATE (B...,+TO NEGOTIATING BANK ONLY: PLEASE FORWARD ALL ...,"CTBC BANK CO., LTD.",SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICA...,CTBC BANK CO LTD,4924,4940
30,1.SIGNED COMMERCIAL INVOICE IN DUPLICATE SHOWI...,1.INSURANCE TO BE COVERED BY APPLICANT.2.CHART...,+THE NEGOTIATING BANK IS REQUESTED TO SEND THE...,CHANG HWA COMMERCIAL BANK LTD.,1SIGNED COMMERCIAL INVOICE IN DUPLICATE SHOWI...,CHANG HWA COMMERCIAL BANK LTD,5345,5374
33,+SIGNED COMMERCIAL INVOICE IN 03 COPIES+FULL S...,+ THE COPY OF SHIPPING DOCUMENTS SUCH AS INVOI...,+REIMBURSE YOURSELVES ON THE REIMBURSING BANK ...,KEB HANA BANK,SIGNED COMMERCIAL INVOICE IN 03 COPIESFULL SE...,KEB HANA BANK,112,125
34,+SIGNED COMMERCIAL INVOICE IN 03 COPIES+FULL S...,+ THE COPY OF SHIPPING DOCUMENTS SUCH AS INVOI...,+REIMBURSE YOURSELVES ON THE REIMBURSING BANK ...,KEB HANA BANK,SIGNED COMMERCIAL INVOICE IN 03 COPIESFULL SE...,KEB HANA BANK,112,125
41,+1.FULL SET OF ORIGINALS AND 2 NON-NEGOTIABLE ...,+ ALL DOCUMENTS TO INDICATE THE NUMBER AND DAT...,DOCUMENTS TO BE DESPATCHED TO STANDARD CHARTER...,STANDARD CHARTERED BANK,1FULL SET OF ORIGINALS AND 2 NON-NEGOTIABLE C...,STANDARD CHARTERED BANK,1597,1620


In [267]:
# 驗證數據
for i in tqdm(range(len(df))):
    assert df['string_X_train'].values[i][df['string_Y_1'].values[i]:df['string_Y_2'].values[i]] == df['Y_label'].values[i]
#df['predict'] = df['string_X_train'].apply(lambda x:get_bank(x,split_keywords=split_keywords,尾綴列表=尾綴列表))
df['predict'] = df['string_X_train'].apply(lambda x:get_bank_寶典(x,銀行列表))

<ipython-input-267-88bf54e70966>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(df))):


In [268]:
df['correct'] = 'no'
df.loc[df.predict==df.Y_label,'correct'] = 'yes'
print('acc:',len(df[df.correct=='yes'])/len(df))

acc: 0.7729831144465291


In [269]:
df.sample(20)

,46A,47A,78,LCBK,string_X_train,Y_label,string_Y_1,string_Y_2,predict,correct
20265,+SIGNED DRAFTS FOR 100 PCT OF INVOICE VALUE.\r...,+COMPLETE NAME AND ADDRESS OF APPLICANT (FIELD...,THE NEGOTIATING BANK IS TO FORWARD ORIGINAL SE...,AXIS BANK LTD,SIGNED DRAFTS FOR 100 PCT OF INVOICE VALUE S...,AXIS BANK LTD,3606,3619,AXIS BANK LTD,yes
6459,1. SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS A...,+ ONE ADDITIONAL COPY/PHOTOCOPY OF ALL REQUIRE...,+ THE AMOUNT AND DATE OF EACH DRAWING MUST BE ...,VIETINBANK,1 SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS A...,VIETINBANK,189,199,VIETINBANK,yes
2629,"DOCUMENTS (IN TRIPLICATE, UNLESS OTHERWISE SPE...",+BILLS OF LADING MUST NOT CONTAIN ANY OF THE F...,+AMOUNT OF DRAWING MUST BE ENDORSED ON THE REV...,CIMB BANK BERHAD,DOCUMENTS IN TRIPLICATE UNLESS OTHERWISE SPEC...,CIMB BANK BERHAD,175,191,CIMB BANK BERHAD,yes
382,THIS DOCUMENTARY CREDIT IS AVAILABLE BY NEGOTI...,1. CHARTER PARTY / TANKER BILLS OF LADING ARE ...,UPON RECEIPT OF DOCUMENTS ISSUED IN STRICT COM...,UBS SWITZERLAND AG,THIS DOCUMENTARY CREDIT IS AVAILABLE BY NEGOT...,UBS SWITZERLAND AG,399,417,UBS SWITZERLAND AG,yes
15461,1. SIGNED COMMERCIAL INVOICE IN 1 COPY INDICAT...,1. 3/3 ORIGINAL B/L TO BE COURIERED TO APPLICA...,1. ALL REQUIRED DOCUMENTS MUST BE SENT DIRECTL...,CATHAY UNITED BANK,1 SIGNED COMMERCIAL INVOICE IN 1 COPY INDICAT...,CATHAY UNITED BANK,118,136,CATHAY UNITED BANK,yes
20669,1. SIGNED COMMERCIAL INVOICES IN 3 COPIES INDI...,1. THE LANGUAGE OF ALL DOCUMENTS MUST BE IN EN...,1. ALL REQUIRED DOCUMENTS MUST BE SENT DIRECTL...,CATHAY UNITED BANK,1 SIGNED COMMERCIAL INVOICES IN 3 COPIES INDI...,CATHAY UNITED BANK,175,193,CATHAY UNITED BANK,yes
7377,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE\r\n+P...,DOCUMENTS MUST BE PRESENTED PRIOR TO CREDIT EX...,T/T REIMBURSEMENT ALLOWED\r\nALL DOCUMENTS MUS...,SHINHAN BANK,SIGNED COMMERCIAL INVOICE IN TRIPLICATE PACK...,SHINHAN BANK,146,158,SHINHAN BANK SEOUL,no
1963,+ SIGNED COMMERCIAL INVOICE IN TRIPLICATE\r\n+...,+A DISCREPANCY FEE OF USD80.00(OR EQUIVALENT) ...,+ALL DOCUMENTS MUST BE FORWARDED DIRECTLY TO O...,INDUSTRIAL BANK OF KOREA SEOUL,SIGNED COMMERCIAL INVOICE IN TRIPLICATE FUL...,INDUSTRIAL BANK OF KOREA SEOUL,957,987,INDUSTRIAL BANK OF KOREA,no
3056,+SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL.\r\n+...,+ALL DOCUMENTS EXCEPT COMMERCIAL INVOICE AND D...,TO NEGOTIATING BANK:\r\n+DISCOUNT CHARGES ARE ...,TAIWAN COOPERATIVE BANK,SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL ITEM...,TAIWAN COOPERATIVE BANK,1889,1912,TAIWAN COOPERATIVE BANK,yes
14826,+SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL(S) AN...,+AN EXTRA COPY OF ALL DOCUMENTS IS REQUIRED FO...,+A DISCREPANCY FEE OF USD70.00(JPY7700.00 OR E...,CHANG HWA COMMERCIAL BANK LTD.,SIGNED COMMERCIAL INVOICE IN 1 ORIGINALS AND ...,CHANG HWA COMMERCIAL BANK LTD,2971,3000,CHANG HWA COMMERCIAL BANK LTD,yes


In [270]:
# 保存
np.save('../data/寶典/銀行寶典.npy',銀行列表)
np.save('../data/寶典/銀行尾綴.npy',尾綴列表)
np.save('../data/寶典/銀行關鍵字.npy',split_keywords)
df.to_csv('../data/preprocess_for_SQUAD_銀行.csv')
print(df.shape)

(3731, 10)
